In [1]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
im = Image.open('chad/1985.jpg')
im.show()

In [2]:
import os
import numpy as np

In [3]:
import os
sat_images = {}

for filename in os.listdir('chad/'):
    if filename.endswith(".jpg"):
        sat_images[filename] = Image.open('chad/' + filename)

In [46]:
sat_images

{'1999.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E240>,
 '1989.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E2B0>,
 '1988.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E320>,
 '2003.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E390>,
 '2002.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E400>,
 '2016.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E198>,
 '2000.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E470>,
 '2014.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E4E0>,
 '2015.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E550>,
 '2001.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E5C0>,
 '2005.jpg': <PIL.JpegImagePlugin.JpegImageFile im

In [48]:

def get_water(sat_images):
    for year in sat_images:
        im_array = np.array(sat_images[year])
    
        sh = im_array.shape()
        y_cent = int(sh[0] / 2)
        x_cent = int(sh[1] / 2)
               
        ref_pix1 = im_array[y_cent][x_cent]
    
        ref_pix2 = im_array[y_cent + 50][x_cent + 50]
    
        ref_pix3 = im_array[y_cent + 50][x_cent - 50]
    
        refs = []
    
        refs.append(ref_pix1, ref_pix2, ref_pix3)

        min_r = 255
        min_g = 255
        min_b = 255
    
        max_r = 0
        max_g = 0
        max_b = 0
    
        for ref in refs:
            if ref[0] < min_r:
                min_r = ref[0]
        
            if ref[1] < min_g:
                min_g = ref[1]
        
            if ref[2] < min_b:
                min_b = ref[2]
            
            if ref[0] > max_r:
                max_r = ref[0]
        
            if ref[1] > max_g:
                max_g = ref[1]
        
            if ref[2] > max_b:
                max_b = ref[2]
            
        for row in im_array:
            for pixel in row:
                if pixel[2] <= max_b and pixel[2] >= min_b and pixel[0] >= min_r and pixel[0] =< max_r and pixel[1] >= min_g and pixel[1] <= max_g:
                    pixel[0] = 0
                    pixel[1] = 0
                    pixel[2] = 245
                
    
    im = Image.fromarray(im_array)
    im.save(year + "processed.jpg")

TypeError: iteration over a 0-d array

In [7]:
year = "2000.jpg"
im_array = np.array(sat_images[year])
print(im_array.shape)

(3251, 4800, 3)
